## Introduction
During the year between May 2014 and May 2015, over 20,000 homes were bought and sold in King County, the most populous county in Washingon state.  The most notable city is Seattle, which in 2015 had a population of about 675,000.  It's home to major tech corporations such as Microsoft and Amazon.  The local government collected data from home purchases and made these data available on their website. This notebook discusses a method for predicting home prices using the nineteen features found in the King County public dataset.  

## The Dataset
The King County Housing Dataset was scraped from King County's public data portal and posted to the data website Kaggle.com by user harlfoxem. It can be found at this link: https://www.kaggle.com/harlfoxem/housesalesprediction/data. Before sharing the data on Kaggle, user hrlfoxem cleaned it to provide users a smooth entry into practicing linear regression techniques.  The dataset has nineteen features, plus the date sold and unique id for each home.  The features are as follows:

|Feature Name|Description|
|-|
|id|unique home identifier|
|date|date house was sold|
|price|price is prediction target|
|bedrooms|number of bedrooms|
|bathrooms|number of bathrooms|
|sqft_living|square footage of the home|
|sqft_lot|square footage of the lot|
|floors|total floors (levels in house|
|waterfront|indicates whether the home is positioned on the water|
|view|indicates whether the home has been viewed|
|condition|rating of the overall condition of the home|
|grade|overall grade given to the housing unit based on King County grading system|
|sqft_above|square footage of house apart from basement|
|sqft_basement|square footage of the basement|
|yr_built|year the home was built|
|yr_renovated|year the home was renovated|
|zipcode|zipcode the home resides in|
|lat|latitude coordinate|
|long|longitude coordinate|
|sqft_living15|living room area in 2015 (implies some renovations) -this might or might not have affected the lotsize area|
|sqft_lot15|lot size area in 2015 (implies some renovations)|

### Analysis


In [ ]:
# import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import matplotlib.ticker as mtick
import seaborn as sns

df = pd.read_csv("kc_house_data.csv")
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [ ]:
# iterate over rows checking for any missing values 
sum([True for idx,row in df.iterrows() if any(row.isnull())])

There are no rows with missing values.

In [ ]:
df.shape

At this time, we have determined that the data needs no more pre-processing. 21,613 different home purchases are available to use in our model. Lets start with a look at the distribution of home prices.

In [ ]:
#investigate price column 
fmt = '${x:,.0f}'
tick = mtick.StrMethodFormatter(fmt)

fig = plt.figure(figsize=(12, 9))

ax1 = fig.add_subplot(2,1,1)
ax1.hist(df.price, bins = 300, color = 'xkcd:grey')
ax1.set_title("Home Prices in King County")
ax1.xaxis.set_major_formatter(tick) 

ax2 = fig.add_subplot(2,1,2)
ax2.hist(df.price, bins = 500, color = 'xkcd:grey')
ax2.set_xlim(0,1000000)
ax2.set_title("Home Prices in King County up to $1,000,000")
ax2.xaxis.set_major_formatter(tick) 

plt.show()


A correlation matrix is a figure that allows us to see the relationship between each of the features in our dataset.  The sns heatmap() function allows us to visaualize with color those variables that are highly correlated to each other.  We'll be focusing specifically on the correlates to price. 

In [ ]:
print(df.corr()['price'].sort_values(ascending = False)[1:11])

In [ ]:
plt.figure(figsize = (20,20))
sns.heatmap(df.corr(),cmap='RdGy',linewidths=0.1,linecolor='white',annot=True)
plt.show()

The top correlates to home price are:

Feature | Correlation
--- | ---
sqft_living | 0.70 <br>
grade | 0.67<br>
sqft_above | 0.61<br>
sqft_living15 | 0.59<br>
bathrooms | 0.53<br>
view | 0.40<br>

There are also strong correlations between the variables that have to do with square footage.  For example, a home with a large square footage is likely to also have a large value for 
square footage upstairs and downstairs.  Also, the number of bedrooms is positively correlated to square footage values.  

Below are scatterplots for each of the top correlates.

In [ ]:
fig = plt.figure(figsize = (20,20))
fmt = '${x:,.0f}'
tick = mtick.StrMethodFormatter(fmt)
#fig.yaxis.set_major_formatter(tick) 

ax1 = fig.add_subplot(3,2,1)
ax2 = fig.add_subplot(3,2,2)
ax3 = fig.add_subplot(3,2,3)
ax4 = fig.add_subplot(3,2,4)
ax5 = fig.add_subplot(3,2,5)
ax6 = fig.add_subplot(3,2,6)

#plot price vs sqft_living
ax1.scatter(df.sqft_above,df.price, c = 'xkcd:black', alpha = .15)
ax1.set_xlabel('Square footage - total')
ax1.set_ylabel('Price')
ax1.yaxis.set_major_formatter(tick) 

#plot price vs grade
ax2.scatter(df.grade,df.price, c = 'xkcd:black', alpha = .15)
ax2.set_xlabel('Grade')
ax2.set_ylabel('Price')
ax2.yaxis.set_major_formatter(tick) 

#plot price vs sqft_above
ax3.scatter(df.sqft_above,df.price, c = 'xkcd:black', alpha = .15)
ax3.set_xlabel('Square footage - above basement')
ax3.set_ylabel('Price')
ax3.yaxis.set_major_formatter(tick) 

#plot price vs sqft_living15
ax4.scatter(df.sqft_living15,df.price, c = 'xkcd:black', alpha = .15)
ax4.set_xlabel('Square footage - living room')
ax4.set_ylabel('Price')
ax4.yaxis.set_major_formatter(tick) 

#plot price vs bathrooms
ax5.scatter(df.bathrooms,df.price, c = 'xkcd:black', alpha = .15)
ax5.set_xlabel('Bathrooms')
ax5.set_ylabel('Price')
ax5.yaxis.set_major_formatter(tick) 

#plot price vs view
ax6.scatter(df.view,df.price, c = 'xkcd:black', alpha = .15)
ax6.set_xlabel('View')
ax6.set_ylabel('Price')
ax6.yaxis.set_major_formatter(tick) 

plt.show()

 Each of the square footage metrics have a weak linear relationship.  The grade and bathrooms features have significant variance depending on each discrete value, and the view seems to not be a good linear fit. 

### More Visualizations

In [ ]:
fig = plt.figure(figsize = (20,15))

renovated = df[df['yr_renovated'] > 0]
not_renovated = df[df['yr_renovated'] <= 0]
to_plot_renovated = [renovated.price, not_renovated.price]
ax1 = fig.add_subplot(3,1,1)
ax1.set_xlabel("Price")
ax1.xaxis.set_major_formatter(tick) 
plt.boxplot(to_plot_renovated, vert = False, widths = .25,labels = ['Renovated Homes','Non-Renovated Homes'])

waterfront = df[df['waterfront'] > 0]
not_waterfront = df[df['waterfront'] <= 0]
to_plot = [waterfront.price, not_waterfront.price]
ax2 = fig.add_subplot(3,1,2)
ax2.set_xlabel("Price")
ax2.xaxis.set_major_formatter(tick)
plt.boxplot(to_plot, vert = False, widths = .25,labels = ['Waterfront Homes','Non-Waterfront Homes'])

view = df[df['view'] > 0]
not_view = df[df['view'] <= 0]
to_plot = [view.price, not_view.price]
ax3 = fig.add_subplot(3,1,3)
ax3.set_xlabel("Price")
ax3.xaxis.set_major_formatter(tick) 
plt.boxplot(to_plot, vert = False, widths = .25,labels = ['Viewed Homes','Non-Viewed Homes'])
plt.show()

Homes that have been renovated tend to have a higher value.  Seventy-five percent of non-renovated homes are less expensive than half of all renovated homes. 

Homes on the waterfront tend to be much more expensive.  A full seventy-five percent of non-waterfront homes are less expensive than the cheapest quarter of all homes on the waterfront. 

Homes that have not been viewed tend to be less valuable than viewed homes.  

# Creating a linear regression model

[Linear Regression](http://en.wikipedia.org/wiki/Linear_regression) is a method to model the relationship between a set of independent variables $X$ (also knowns as explanatory variables, features, predictors) and a dependent variable $Y$.  This method assumes the relationship between each predictor $X$ is **linearly** related to the dependent variable $Y$. Multiple linear regression contains multiple independent variables.

$$ Y = \beta_0 + \beta_1 X_1 + \ldots + \beta_p X_p + \epsilon$$ 

In [ ]:
# import packages
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from statsmodels.graphics.regressionplots import *
from scipy import stats

# this function returns the mean absolute percentage error between predicted and actual values
def mean_absolute_percentage_error(y_test, y_pred):
    # return the MAPE score
    return np.mean(np.abs((y_test - y_pred) / y_test)) * 100

# this function performs linear regression on a data set
# input: data and target
def make_linear_model(X, y, plot = True):
    # create train and test sets with 70/30 split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 23)
    
    # fit the train data and create predictions with test data
    reg = linear_model.LinearRegression()
    reg.fit(X_train, y_train)
    y_pred = reg.predict(X_test)
    
    # create dataframe with features and coefficients
    results = pd.DataFrame({'Features': X.columns, 'Estimated Coefficients': reg.coef_})[['Features', 'Estimated Coefficients']]
    
    # print results and summary statistics
    print(results)
    print('\nEstimated intercept coefficient: {}'.format(reg.intercept_))    
    print('\nSummary Statistics\nR-squared value: {}'.format(reg.score(X_test, y_test)))
    print("Root Mean Squared Error: {}".format(np.sqrt(mean_squared_error(y_test,y_pred))))
    print("Mean Absolute Percentage Error (MAPE): {}".format(mean_absolute_percentage_error(y_test,y_pred)))
 
    if plot == True:        
        
        fig = plt.figure(figsize = (18,5))    
        # plot residuals
        ax1 = fig.add_subplot(1,3,1)
        plt.scatter(y_pred, y_pred - y_test, c = 'xkcd:grey', marker = '.', s = 40)
        plt.hlines(y=0, xmin = -50000, xmax = 3750000)
        ax1.set_xlim(0,y_pred.max())
        ax1.tick_params(rotation = 35)
        ax1.xaxis.set_major_formatter(tick) 
        ax1.yaxis.set_major_formatter(tick)
        ax1.set_title('Residual plot using test data')
        ax1.set_ylabel('Residuals')

        # plot probability plot 
        ax2 = fig.add_subplot(1,3,2)  
        stats.probplot(y_pred - y_test, plot = plt)


        # plot true prices vs predicted prices
        # fig2, ax3 = plt.subplots(figsize = (8,7))
        ax3 = fig.add_subplot(1,3,3)
        plt.scatter(y_test, y_pred,c = 'xkcd:grey', alpha = .25)
        ax3.set_xlabel("True prices")
        ax3.set_ylabel("Predicted Prices")
        ax3.xaxis.set_major_formatter(tick) 
        ax3.yaxis.set_major_formatter(tick)
        ax3.set_title("True prices compared to predicted prices")
        ax3.tick_params(rotation = 35)
        ax3.set_xlim(0,y_pred.max())
        ax3.set_ylim(0,y_pred.max())
        plt.plot([0, y_pred.max()], [0, y_pred.max()],c = 'xkcd:black',linestyle='-', linewidth=2)

        plt.subplots_adjust(wspace = .35)
        plt.show()

## Model using all features
The python package StatsModels gives a nice output with many statistics, so we'll use it to gain an overall understanding of the features we're working with.


In [ ]:
X = df.drop(['price','id','date'], axis = 1)
y = df['price'].values

est = sm.OLS(y, X)
fitted = est.fit()
print(fitted.summary())

Looking at the output, we can already see that the feature "floors" is not a useful indicator of home price, due to it's high P-value.  We'll remove it from the feature space.

## Evaluating Success
Our goal is to create a model that can accurately forecast home prices.  One common metric we can use is the mean absolute percentage error (MAPE).  It quantifies how far off the model is in absolute terms.  Going forward, I'll be using the MAPE score to assess the quality of each model.  

In [ ]:
make_linear_model(X,y)

Using my custom function, we can see that the MAPE score using all features is __25.83%__.  This is our baseline that we will attempt to improve upon.  From the plots above, we can see that the residuals are not evenly distributed, which indicates some non-linear factors.  Also, the probability plot shows some skewedness on both ends. 

### Clustering
In order to minimize the MAPE, we'll first cluster the data before applying a model. We'll cluster the data only on the most impactful features, price, sqft_living, and grade.

In [ ]:
from sklearn.cluster import KMeans

df_cluster = df[['price','sqft_living','grade']]

# initalize a KMeans instance and fit it to the dataframe
km = KMeans(n_clusters = 3, init = 'k-means++', n_init = 10)
km.fit(df_cluster)
x = km.fit_predict(df_cluster)

# create new column indicating which cluster the observation belongs to
df['cluster'] = x

# split dataframe based on clustering
# from doing the clustering many times, I determined that the clusters will be split into roughly 15,000, 6,000, and 600 
# observations.  To remain consistent, I'll assign them titles in the order of
# cluster 0 being the smallest group and cluster2 being the largest.

if len(df.loc[df.cluster == 0].index) < 1000:
    cluster0 = df.loc[df.cluster == 0]
elif len(df.loc[df.cluster == 1].index) < 1000:
    cluster0 = df.loc[df.cluster == 1]
elif len(df.loc[df.cluster == 2].index) < 1000:
    cluster0 = df.loc[df.cluster == 2]
    
if (len(df.loc[df.cluster == 0].index) > 1000) & (len(df.loc[df.cluster == 0].index) < 10000):
    cluster1 = df.loc[df.cluster == 0]
elif (len(df.loc[df.cluster == 1].index) > 1000) & (len(df.loc[df.cluster == 1].index) < 10000):
    cluster1 = df.loc[df.cluster == 1]   
elif (len(df.loc[df.cluster == 2].index) > 1000) & (len(df.loc[df.cluster == 2].index) < 10000):
    cluster1 = df.loc[df.cluster == 2]   
    
if len(df.loc[df.cluster == 0].index) > 10000:
    cluster2 = df.loc[df.cluster == 0]
elif len(df.loc[df.cluster == 1].index) > 10000:
    cluster2 = df.loc[df.cluster == 1]
elif len(df.loc[df.cluster == 2].index) > 10000:
    cluster2 = df.loc[df.cluster == 2]

print(cluster0.shape)
print(cluster1.shape)
print(cluster2.shape)

### Visualizing the Clusters
We can see that the data is now clustered into three groups, of varied size.  Below is a look at the distribution of the prices of the clusters.

In [ ]:
to_plot = [cluster0.price, cluster1.price, cluster2.price]

fig = plt.figure(figsize = (20,15))
ax = fig.add_subplot(3,1,1)
ax.set_xlabel("Price")
ax.xaxis.set_major_formatter(tick) 
plt.boxplot(to_plot, vert = False, widths = .25,labels = ['Cluster 0','Cluster 1','Cluster 2'])

to_plot = [cluster0.sqft_living, cluster1.sqft_living, cluster2.sqft_living]

ax2 = fig.add_subplot(3,1,2)
ax2.set_xlabel("Square Footage of Home")
plt.boxplot(to_plot, vert = False, widths = .25,labels = ['Cluster 0','Cluster 1','Cluster 2'])

to_plot = [cluster0.grade, cluster1.grade, cluster2.grade]

ax3 = fig.add_subplot(3,1,3)
ax3.set_xlabel("Grade")
plt.boxplot(to_plot, vert = False, widths = .25,labels = ['Cluster 0','Cluster 1','Cluster 2'])
plt.show()

Cluster 0, which has about 600 observations, seems to be a grouping of the higher priced homes.  The homes in this group tend to have higher square footage, and grade values as well. There are several outliers at the high end, but most prices fall between about $1.5 and 3 million.  

Cluster 1 has about 6,000 observations, and contains homes priced at about $500,000 to the lower bounds of Cluster 0. The square footage and grade are in the middle of clusters 0 and 2.

Cluster 2 has the most observations, with 15,000.  It's comprised of the lowest priced homes, with the smallest square footage, and mostly lower grades.  It makes sense that most homes in this cluster are sold for under a million dollars.

## Feature Encoding

To improve the predictive power of my model, I'm using feature encoding. To eliminate some of the variability of a feature, we can group each observation into intervals of similarly priced homes.  After each observation is in it's bucket, we calculate the average of each bucket, and then assign that average value to a new column in the dataframe.  In a way, it's similar to clustering, but using only one specific variable.  

For example, when I encode the feature 'sqft_living', each home will be grouped into 30 equally populated buckets corresponding to their square footage.  I'll calculate the average of each bucket, and it will be assigned to a new feature, which can be used to predict the price. 

In [ ]:
def encode_feature(df, feature_name, num_buckets = 30):
    
    # create name for new feture, and copy the dataframe
    bucket = feature_name + '_bucket_interval'
    bucket_price = feature_name + '_bucket_price'
    df_copy = df.copy()
        
    # create a new feature that contains the interval to which the observation belongs
    df_copy[bucket] = pd.qcut(df[feature_name],num_buckets, duplicates = 'drop')
   
    # group the intervals together and calculate the mean price for each interval/bucket
    df_new = df_copy.groupby(df_copy[bucket], as_index = False).agg({'price':'mean'})
    df_new = pd.DataFrame(df_new)
    
    # specify the column names, and create a row that contains the number of the bucket for plotting
    df_new.rename(columns = {'price':bucket_price}, inplace = True)
    df_new['bucket_number'] = df_new.index + 1
    df_new.head()
    
    # show a scatter plot of the buckets
    fig, ax = plt.subplots(figsize = (8,6))
    plt.scatter(df_new.bucket_number,df_new[bucket_price])
    plt.xlabel('Bucket Number - ' + feature_name)
    plt.ylabel('Average price')
    ax.yaxis.set_major_formatter(tick) 
    plt.show()
    
    #join df and df_new on the new bucket_interval feature
    df_join = pd.merge(df_new, df_copy, on=bucket)
    df_join.drop([bucket, 'bucket_number'], axis = 1, inplace = True)
    return df_join

In [ ]:
# create encoded features
cluster0 = encode_feature(cluster0,'sqft_living')
cluster0 = encode_feature(cluster0, 'grade')
cluster0 = encode_feature(cluster0, 'bathrooms')

From the graphs, the encoded features for sqft_living, grade, and bathrooms seem to have a quadratic relationship with price.  No other variables showed clear relationships after encoding.  In order to capture the quadratic relationship, we'll create new features by squaring the encoded value for each observation.  

In [ ]:
cluster0['sqft_living_bucket_price_squared'] = cluster0['sqft_living_bucket_price'] ** 2
cluster0['grade_bucket_price_squared'] = cluster0['grade_bucket_price'] ** 2
cluster0['bathrooms_bucket_price_squared'] = cluster0['bathrooms_bucket_price'] ** 2

Before creating a model for Cluster 0, we will look at the top correlates to guide our feature selection.

In [ ]:
# print the top ten correlates to price -  index 1-11 because index 0 is price itself, which has a correlation of 1.
print(cluster0.corr()['price'].sort_values(ascending = False)[1:11])

In [ ]:
features = ['sqft_living_bucket_price_squared','waterfront','condition','grade', 'sqft_living']
X = cluster0[features]
y = cluster0.price

make_linear_model(X,y)

A MAPE score of __17.36%__ is a vast improvement over our original score of __25.85%__.  The distribution of the residuals is somewhat normal, but there are signs of skewedness at the edges as seen in the probability plot.  The table below describes all combination of features tested with their corresponding MAPE scores.

Features Used | MAPE
--- | ---
sqft_living_bucket_price_squared, waterfront, condition, grade, sqft_living | 17.360 <br>
sqft_living_bucket_price_squared, bathrooms, waterfront, condition, grade, sqft_living | 17.365 <br>
sqft_living_bucket_price_squared, bathrooms, waterfront, condition, grade, yr_built, sqft_living | 17.477<br>
sqft_living_bucket_price_squared, bathrooms, waterfront, condition, grade, yr_built, sqft_living, sqft_living15 | 17.506<br>
bathrooms_bucket_price, grade_bucket_price_squared,  sqft_living_bucket_price_squared, bathrooms, waterfront, condition, grade, yr_built, sqft_living, sqft_living15 | 17.686<br>
sqft_above, bathrooms_bucket_price, grade_bucket_price_squared, sqft_living_bucket_price_squared, bathrooms, waterfront, condition, grade, yr_built, sqft_living, sqft_living15 | 17.623<br>
sqft_above, bathrooms_bucket_price_squared, grade_bucket_price_squared, sqft_living_bucket_price_squared, bathrooms, waterfront, condition, grade, yr_built, sqft_living, sqft_living15 | 17.683<br>
bathrooms_bucket_price_squared, grade_bucket_price_squared, sqft_living_bucket_price_squared, bathrooms, waterfront, condition, grade, yr_built, sqft_living, sqft_living15 | 17.705<br>
grade_bucket_price_squared,  sqft_living_bucket_price_squared, bathrooms, waterfront, condition, grade, yr_built, sqft_living, sqft_living15 | 17.743<br>
sqft_living_bucket_price_squared, sqft_living, bathrooms_bucket_price, bathrooms, grade_bucket_price_squared, sqft_above, grade | 19.19<br>

To ensure the features we selected are statistically signifcant, we'll run a statsmodels linear regression and look at the P-values for each feature.  

In [ ]:
est = sm.OLS(y, X)
fitted = est.fit()
print(fitted.summary())

The column labeled 'P>|t|' shows us that each of the features we selected are statistically significant influencers of price.  We can be satisfied with this model. 

# Cluster 1


In [ ]:
# create encoded features
cluster1 = encode_feature(cluster1,'sqft_living')
cluster1 = encode_feature(cluster1, 'grade')
cluster1 = encode_feature(cluster1, 'bathrooms')

From the graphs, the encoded features for sqft_living, grade, and bathrooms seem to have either a linear or a quadratic relationship with price.  No other variables showed clear relationships after encoding.  In order to capture the quadratic relationship, we'll create new features by squaring the encoded value for each observation.  

In [ ]:
cluster1['sqft_living_bucket_price_squared'] = cluster1['sqft_living_bucket_price'] ** 2
cluster1['grade_bucket_price_squared'] = cluster1['grade_bucket_price'] ** 2
cluster1['bathrooms_bucket_price_squared'] = cluster1['bathrooms_bucket_price'] **2

Before creating a model for Cluster 1, we will look at the top correlates to guide our feature selection.

In [ ]:
# print the top ten correlates to price -  index 1-11 because index 0 is price itself, which has a correlation of 1.
print(cluster1.corr()['price'].sort_values(ascending = False)[1:11])

In [ ]:
features = ['bathrooms_bucket_price_squared','grade_bucket_price','bathrooms','waterfront','condition','grade','yr_built','zipcode','lat','long', 'sqft_living','sqft_living15']
X = cluster1[features]
y = cluster1.price

make_linear_model(X,y)

A MAPE score of __12.94%__ is a vast improvement over our original score of __25.85%__.  The distribution of the residuals is troubling, but the diagonal pattern seen is caused by the clustering.  This cluster seems to have a strict cut-off point near $600,000.  This causes our probability plot to show a wave-like pattern.  Because this is caused by the clustering, I'm less concerned about it than I otherwise would be.  The table below describes all combination of features tested with their corresponding MAPE scores.

Features Used | MAPE
--- | ---
bathrooms_bucket_price, grade_bucket_price, sqft_living_bucket_price_squared, bathrooms, waterfront, condition, grade, yr_built, zipcode, lat, long, sqft_living, sqft_living15 | 13.116 <br>
bathrooms_bucket_price, grade_bucket_price_squared, sqft_living_bucket_price_squared, bathrooms, waterfront, condition, grade, yr_built, zipcode, lat, long, sqft_living, sqft_living15 | 13.120 <br>
bathrooms_bucket_price_squared, grade_bucket_price, bathrooms, waterfront, condition, grade, yr_built, zipcode, lat, long, sqft_living, sqft_living15 | 13.123 <br>
bathrooms_bucket_price_squared, grade_bucket_price, sqft_living_bucket_price_squared, bathrooms, waterfront, condition, grade, yr_built, zipcode, lat, long, sqft_living, sqft_living15 | 13.123 <br>
grade_bucket_price_squared, sqft_living_bucket_price_squared, bathrooms, waterfront, condition, grade, yr_built, zipcode, lat, long, sqft_living, sqft_living15 | 13.214 <br>
bathrooms_bucket_price, grade_bucket_price_squared, sqft_living_bucket_price_squared, bathrooms, waterfront, condition, grade, yr_built, zipcode, lat, long, sqft_living | 13.270 <br>
bathrooms_bucket_price, grade_bucket_price_squared, sqft_living_bucket_price_squared, bathrooms, waterfront, condition, grade, yr_built, sqft_living, sqft_living15 | 13.941 <br>
bathrooms_bucket_price_squared, sqft_living, grade_bucket_price_squared, grade, sqft_above, bathrooms_bucket_price, bathrooms | 14.67 <br>

To ensure the features we selected are statistically signifcant, we'll run a statsmodels linear regression and look at the P-values for each feature.  

In [ ]:
est = sm.OLS(y, X)
fitted = est.fit()
print(fitted.summary())

The column labeled 'P>|t|' shows us that each of the features we selected are statistically significant influencers of price, since they all fall below the arbitrary value of .05.  We can be satisfied with this model. 

# Cluster 2


In [ ]:
# encode features
cluster2 = encode_feature(cluster2,'sqft_living')
cluster2 = encode_feature(cluster2, 'grade')
cluster2 = encode_feature(cluster2, 'bathrooms')

From the graphs, the encoded features for sqft_living, grade, and bathrooms seem to have either a linear or a quadratic relationship with price.  Grade may be related by the root function.  No other variables showed clear relationships after encoding.  In order to capture the quadratic relationship, we'll create new features by squaring the encoded value for each observation.  

In [ ]:
cluster2['sqft_living_bucket_price_squared'] = cluster2['sqft_living_bucket_price'] ** 2
cluster2['grade_bucket_price_sqrt'] = cluster2['grade_bucket_price'] ** .5

In [ ]:
# print the top ten correlates to price -  index 1-11 because index 0 is price itself, which has a correlation of 1.
print(cluster2.corr()['price'].sort_values(ascending = False)[1:11])

In [ ]:
features = ['bedrooms', 'grade_bucket_price_sqrt', 'sqft_living_bucket_price_squared', 'bathrooms', 'waterfront', 'condition', 'grade', 'yr_built', 'zipcode', 'lat', 'long', 'sqft_living' ]
X = cluster2[features]
y = cluster2.price

make_linear_model(X,y)

A MAPE score of __18.03%__ is a vast improvement over our original score of __25.85%__.  The distribution of the residuals is normal, and as before, the diagonal pattern is caused by the clustering. The probability plot shows us that our residuals are normal. The table below describes all combination of features tested with their corresponding MAPE scores.

 Features Used | MAPE
--- | ---
grade_bucket_price, sqft_living_bucket_price, bedrooms, bathrooms, waterfront, condition, grade, yr_built, zipcode, lat, long, sqft_living | 17.913 <br>
bathrooms_bucket_price, grade_bucket_price, sqft_living_bucket_price, bedrooms, bathrooms, waterfront, condition, grade, yr_built, zipcode, lat, long, sqft_living | 17.913 <br>
bedrooms, grade_bucket_price_sqrt, sqft_living_bucket_price_squared, bathrooms, waterfront, condition, grade, yr_built, zipcode, lat, long, sqft_living |  17.926 <br>
bedrooms, grade_bucket_price, sqft_living_bucket_price_squared, bathrooms, waterfront, condition, grade, yr_built, zipcode, lat, long, sqft_living |  17.927 <br>
grade_bucket_price, sqft_living_bucket_price, bathrooms, waterfront, condition, grade, yr_built, zipcode, lat, long, sqft_living | 18.055 <br>
grade_bucket_price, sqft_living_bucket_price, bathrooms, waterfront, condition, grade, yr_built, lat, long, sqft_living, | 18.066 <br>
lat, grade_bucket_price_sqrt, grade, sqft_living_bucket_price_squared, sqft_living, sqft_above | 18.62 <br>


To ensure the features we selected are statistically signifcant, we'll run a statsmodels linear regression and look at the P-values for each feature.  

In [ ]:
est = sm.OLS(y, X)
fitted = est.fit()
print(fitted.summary())

The column labeled 'P>|t|' shows us that each of the features we selected are statistically significant influencers of price, since they all fall below the arbitrary value of .05.  We can be satisfied with this model. 

# Conclusion

After clustering the data and encoding features, I used the mean absolute percentage error to determine my selected features.  The average improvement after clustering and feature encoding is 9.28%.

# Limitations
This data was collected for 1 year in King County, Washington.  It would be dangerous to attempt to use this model to predict anything other than a home selling in that area during that time period.  

# Recommendations
For clients wishing to purchase a home in King County, Washington, they should be aware that the largest factor when determining cost is the square footage, location, and condition (grade) of the home.  At best, only about 60% of the variance is explained by the model, so the predictions should not be heavily relied upon.